# Atelier ML - Reconnaissance de chiffres manuscrits

Dans cet atelier, vous allez entraîner un modèle qui permet de reconnaitre les chiffres écrits à la main à partir du dataset MNIST. Ensuite, vous pourrez intégrer ce modèle dans une application qui permettra à l'utilisateur de "dessiner" un chiffre et d'afficher la prédiction.

Etapes : 
*   Sélection d'un dataset restreint pour entraîner votre modèle
*   Gestion d'un dataset contenant des images en format pixel
*   Entrainement d'un modèle de Support Vector Machine (SVM)
*   Création d'un Pipeline
*   Sauvegarde d'un modèle avec Pickle
*   Création d'une application Streamlit avec une fonctionnalité de dessin (canva)
*   Prédictions directement dans l'application

In [ ]:
# IMPORTS

Commencez par importer le dataset MNIST et comprendre son contenu.

Vous en savoir plus sur cette base de données : http://yann.lecun.com/exdb/mnist/ 

In [ ]:
df =

Sous quelle forme se présentent vos données ? Que représentent-elles ?

Pour en savoir plus sur les valeurs du dataframe, <a href='https://www.whydomath.org/node/wavlets/imagebasics.html'> petite explication sur l'échelle des couleurs</a>.

Et en exécutant le code ci-dessous, vous y verrez déjà un peu plus clair 😋 

In [ ]:
for i in (np.random.randint(0,270,4)):
    img_28_28 = np.array(df.iloc[i, 1:]).reshape(28,28)
    plt.title('label: {0}'. format(df.iloc[i, 0]))
    plt.imshow(img_28_28, interpolation='antialiased', cmap='gray')
    plt.show()

Ci-dessous une 2ème manière de visualiser le contenu d'une ligne de notre dataframe. Qu'obtenez-vous ?

In [ ]:
for row in img_28_28:
    print(' '.join('{:3}'.format(value) for value in row))

Le site du MNIST donne plus d'informations sur le traitement mené sur les images d'origine pour qu'elles aient des formats comparables :

> The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image by computing the center of mass of the pixels, and translating the image so as to position this point at the center of the 28x28 field.

La description est technique mais permet de comprendre que les images qu'on va dessiner nous-mêmes vont également devoir passer par ce même traitement pour être comparables et donc bien prédites par notre modèle.

## Sélection des données

On a actuellement un dataframe avec 60 000 lignes. Comme vous allez entraîner des modèles, il est utile de réduire la quantité de données qu'on va utiliser ici. Cela risque de réduire la qualité de prédiction mais permet aussi de réduire le temps d'entraînement de nos modèles. Créez un dataframe plus léger en ne gardant que 10 000 lignes du dataframe initial.

Essayez d'avoir une représentation homogène de tous les labels que vous allez prédire (soit les chiffres de 0 à 9).

## Machine Learning

Maintenant, c'est le moment de passer aux choses sérieuses ! On va préparer les données pour le Machine Learning. 

Après avoir défini vos variables explicatives et la variable cible, divisez vos données en jeux d'entraînement et de test.

Vérifiez la distribution des labels dans vos deux jeux (par ex. avec un <a href="https://seaborn.pydata.org/generated/seaborn.countplot.html">counplot</a>).

Les deux jeux ont-ils des distributions de classe similaires ? Sinon, avez-vous pensé à inclure un paramètre dans le train_test_split qui peut y remédier ? 

#### Feature Scaling

Vous pouvez aussi standardiser vos données.

#### PCA

Pour améliorer nos performances et réduire le bruit dans nos données, on peut réduire le nombre de variables utilisées pour notre ML en faisant une PCA. Faites plusieurs tests pour trouver un nombre de composants qui permet de garder une variance expliquée importante avec un bon score de précision tout en réduisant le temps d'entraînement.

### SVM

Super ! Vos données sont prêtes. Vous allez pouvoir entraîner un modèle de Support Vector Machine (SVM). 🤖

Pour séparer les classes, ce modèle génère un hyperplan dans un espace à N-dimensions (autant de dimensions que vous avez de variables explicatives) en maximisant l'espace entre l'hyperplan et les support vecteurs (soit les coordonnées des points).



<img src="https://miro.medium.com/max/1400/1*ZpkLQf2FNfzfH4HXeMw4MQ.png" />


Quelques ressources pour en apprendre plus sur le SVM : 
*   https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/ 
*   https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47 
*   Le modèle est disponible dans <a href="https://scikit-learn.org/stable/modules/svm.html">sklearn</a>.


Entraînez un modèle en testant plusieurs hyperparamètres (par ex. "kernel" pour SVM ou "n_components" pour la PCA). <a href="https://towardsdatascience.com/how-to-select-the-best-number-of-principal-components-for-the-dataset-287e64b14c6d"> Ici un article utile</a> sur la manière dont on peut choisir le nombre de composantes.

Et pourquoi ne pas réaliser une GridSearch pour trouver les meilleurs paramètres ?

Une fois le meilleur modèle identifié, vous pouvez le ré-entrainer en incluant le paramètre `probability=True` : ce paramètre permet d'avoir les probabilités de chaque classe dans les prédictions, ce qui sera utile dans la suite de notre travail.
 
Créez une variable `y_pred` avec les prédictions sur votre jeu de test.

Maintenant que vous avez entrainé votre modèle, on va le tester en pratique sur les images.

Comme au début, on peut afficher plusieurs images de notre jeu de test avec la prédiction faite par notre modèle. Attention, il faut bien distinguer le `X_test` initial qui contient les 748 champs avec les valeurs des pixels, et notre `X_test` transformé après la PCA dont on ne peux plus distinguer les composantes (le code ci-dessous ne fonctionnera que si vous n'avez pas modifié le `X_test` initial mais créé une nouvelle variable en faisant la PCA).

In [ ]:
for i in (np.random.randint(0,270,6)):
    two_d = (np.reshape(X_test.values[i], (28, 28))).astype(np.uint8)
    plt.title('predicted label: {0}'. format(y_pred[i]))
    plt.imshow(two_d, interpolation='antialiased', cmap='gray')
    plt.show()

Affichez une matrice de confusion. Les résultats vous semblent-ils satisfaisants ?

## Création d'une pipeline

Examinez le code ci-dessous puis exécutez-le. 

In [30]:
from sklearn.model_selection import train_test_split

X = df_sample.iloc[:, 1:]
y = df_sample['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC

steps = [('pca', PCA(n_components=.8)), ('svm', SVC(kernel='rbf', probability=True))]
pipeline = Pipeline(steps)    

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
pipeline.predict(X_test)

In [ ]:
print(pipeline.score(X_train, y_train))
print(pipeline.score(X_test, y_test))

On crée ici un Pipeline, il s'agit d'une suite d'actions qui permet en une seule exécution de réaliser plusieurs traitements à la suite sur nos données. 

Ce pipeline reprend ce qu'on a fait plus haut, à savoir il applique la PCA puis le modèle SVM sur nos données. Si vous avez standardisé vos données, vous pouvez ajouter cette étape dans le pipeline. 

Un pipeline peut aussi être utilisé pour tester divers hyperparamètres sur plusieurs étapes d'entraînement du modèle (par ex. n_components dans la PCA, hyperparamètres du modèle) : un exemple concret <a href="https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html"> ici</a>.

## Sauvegarde du modèle

Pour pouvoir rapidement prédire un label sur de nouvelles données (soit des chiffres qu'on va dessiner nous-mêmes !), il est utile de stocker notre modèle entraîné dans un fichier à partir duquel on pourra directement le charger dans notre application, et ainsi ne pas devoir ré-entraîner notre modèle à chaque fois. 

Pour cela, on utilise la librairie Pickle qui permet de 'sérialiser' et de 'désérialiser' des objets Python, c'est-à-dire de stocker dans un format binaire le contenu d'un objet (une liste, un modèle...) qu'on peut ré-importer ensuite.

*   Documentation Python sur Pickle (FR) : https://docs.python.org/fr/3/library/pickle.html
*   https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
*   https://www.quennec.fr/trucs-astuces/langages/python/python-le-module-pickle 

In [35]:
import pickle
filename = 'digit_classifier.sav'

In [36]:
# Save the model
pickle.dump(pipeline, open(filename, 'wb'))

Après l'exécution de ce code, vous devriez avoir un nouveau fichier `digit_classifier.sav` qui stocke votre modèle.

In [37]:
 # Load the model
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
loaded_model

## Test des prédictions

On va tester notre modèle enregistré sur une des prédictions (1e ligne de X_test).

In [ ]:
# On a 784 colonnes correspondant aux 28x28 pixels de l'image
img_test_0 = X_test.iloc[0].values
img_test_0.shape

In [ ]:
# On ajoute une dimension pour la prédiction
img_test_0 = np.array(img_test_0).reshape(1, -1) 
img_test_0.shape

Affichez la valeur prédite pour la 1ère ligne de X_test.

In [ ]:
# On change les dimensions de la 1e ligne pour obtenir une matrice de 28x28
img_test_0 = img_test_0.reshape(28, 28)
img_test_0.shape

On peut ainsi afficher la matrice au format 28x28. Le chiffre prédit apparait-il bien dans cette matrice ?


In [ ]:
for row in img_test_0:
    print(' '.join('{:3}'.format(value) for value in row))

On va maintenant tenter d'utiliser notre modèle pour prédire un chiffre à partir d'une image qu'il ne connait pas. 

On teste avec une image sauvegardée (`image.jpg`) dont on change la taille au format voulu et ré-enregistre.

In [ ]:
image = Image.open("image.jpg")
size = 28, 28

print(image.size)
image.thumbnail(size,Image.ANTIALIAS)
print(image.size)
image.save("image_28px.jpg")

On visualise comme avant la nouvelle image.

In [ ]:
# Source: https://stackoverflow.com/questions/40727793/how-to-convert-a-grayscale-image-into-a-list-of-pixel-values 

img = Image.open('image_28px.jpg').convert('L')  # convert image to 8-bit grayscale

data = list(img.getdata()) # convert image data to a list of integers
# convert that to 2D list (list of lists of integers)
data = [data[offset:offset+28] for offset in range(0, 28*28, 28)]

for row in data:
    print(' '.join('{:3}'.format(value) for value in row))

On va maintenant pouvoir mettre cette image dans le même format que les images du dataset d'entraînement, en reprenant le code de <a href='https://medium.com/@o.kroeger/tensorflow-mnist-and-your-own-handwritten-digits-4d1cd32bbab4'> cet article</a>. 

Il n'est pas nécessaire de comprendre précisément tout le code mais seulement les grandes étapes de ce qu'il permet d'obtenir.

In [ ]:
gray = np.array(data)
gray

In [ ]:
# On remplace les valeurs basses par des 0
gray = np.where(gray > 50, gray, 0)

for row in gray:
    print(' '.join('{:3}'.format(value) for value in row))

In [71]:
# remove every row and column at the sides of the image which are completely black.

while np.sum(gray[0]) == 0:
    gray = gray[1:]

while np.sum(gray[:,0]) == 0:
    gray = np.delete(gray,0,1)

while np.sum(gray[-1]) == 0:
    gray = gray[:-1]

while np.sum(gray[:,-1]) == 0:
    gray = np.delete(gray,-1,1)

rows,cols = gray.shape

In [72]:
gray = gray.astype('float32')

In [ ]:
gray.shape

In [74]:
#!pip install opencv-python
import cv2

In [75]:
# Now we want to resize our outer box to fit it into a 20x20 box. We need a resize factor for this.

if rows > cols:
    factor = 20.0/rows
    rows = 20
    cols = int(round(cols*factor))
    gray = cv2.resize(gray, (cols,rows))
else:
    factor = 20.0/cols
    cols = 20
    rows = int(round(rows*factor))
    gray = cv2.resize(gray, (cols,rows))

In [76]:
# As we need a 28x28 pixel image, we add the missing black rows and columns using the np.lib.pad function which adds 0s to the sides.

import math

colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')

In [77]:
# Using the center of mass (function 1), we shift the image so that it is centered (function 2)

from scipy import ndimage

def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

In [ ]:
# Applying the functions

shiftx,shifty = getBestShift(gray)
shifted = shift(gray,shiftx,shifty)
gray = shifted

In [ ]:
gray

In [ ]:
# Conversion au format adapté
img_flattened = gray.flatten()
img_flattened_784 = np.array(img_flattened).reshape(1, -1) 
img_flattened_784.shape

In [ ]:
# On affiche l'image en pixel comme avant

data = img_flattened_784.reshape(28, 28)
data = data.astype('int64')

for row in data:
    print(' '.join('{:3}'.format(value) for value in row))

Au vu des valeurs des pixels, notre traitement semble avoir marché, le chiffre est désormais bien centré.

Vous pouvez maintenant prédire la valeur dans l'image ! Votre modèle trouve-t-il le bon chiffre ?

Avec la méthode <a href= "https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict_proba">predict_proba</a>, on peut aussi obtenir la probabilité associée à chaque chiffre (soit les classes prédites). Quel est le degré de certitude de votre prédiction ? Si le modèle a mal prédit le chiffre, la deuxième prédiction est-elle bonne ?

## Mise en pratique : application à des nouvelles images dessinées en direct

### Fonctions

Voie rapide : le script Python `classifier.py` contient plusieurs fonctions qui permettent d'appliquer toutes les étapes qu'on vient de mener à de nouvelles données. 

Voie longue mais plus ambitieuse : vous pouvez aussi faire ce script vous-mêmes, par exemple créer une fonction qui applique à une image fournie en entrée le formattage requis pour réaliser la prédiction et une deuxième fonction qui peut ensuite appliquer le modèle enregistré à l'image formattée. 

### Application Streamlit

Ensuite, exécutez le script app.py avec la commande `streamlit run app.py`. Pour cela vous devrez sûrement installer les librairies `streamlit` et `streamlit_drawable_canvas`. 

Que fait ce script ?  🤔 

En faisant appel aux fonctions que vous avez définies plus tôt, modifiez le script pour pouvoir afficher votre prédiction lorsqu'on dessine un chiffre sur le canvas. 

Pour aller plus loin, vous pouvez maintenant : 
*   afficher le pourcentage de certitude ou encore le deuxième résultat lorsque la prédiction est incertaine ou incorrecte
*   stocker les nouvelles images dessinées à la main et recréer votre propre base de données
*   découvrir comment entrainer un réseau de neurones pour faire la même chose (comme <a href="https://data-flair.training/blogs/python-deep-learning-project-handwritten-digit-recognition/">ici</a> et <a href="https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/">ici</a> par exemple) et appliquer ce modèle aux images dessinées dans votre app. Mais cela vous demandera beaucoup plus de recherche et d'investissement 😉


Autres sources
*   https://www.kaggle.com/datasets/oddrationale/mnist-in-csv?select=mnist_train.csv 
*   https://towardsdatascience.com/support-vector-machine-mnist-digit-classification-with-python-including-my-hand-written-digits-83d6eca7004a 